In [7]:
import sys
sys.path.append('./models/')

import torch
import torch.nn.functional as F
import numpy as np
import sounddevice as sd # 마이크 입력을 위한 라이브러리

In [8]:
sd.default.device = (9, None)

In [9]:
MODEL_PATH = "./result/best.pt"
CLASS_NAME = ["danger", "fire", "gas", "non", "tsunami"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SAMPLE_RATE = 16000
# DURATION = 1
NUM_SAMPLES = 8000

In [10]:
def preprocess(audio_np):
    # numpy 배열을 torch 텐서로 변환, 배치 차원 추가: (1, num_samples)
    waveform = torch.tensor(audio_np, dtype=torch.float32).unsqueeze(0)

    # 절대값 정규화
    waveform = waveform / (waveform.abs().max() + 1e-9)

    # 길이가 부족할 경우 뒤에 0을 채움
    if waveform.shape[1] < NUM_SAMPLES:
        waveform = F.pad(waveform, (0, NUM_SAMPLES - waveform.shape[1]))
    # 길이가 길 경우 자름
    else:
        waveform = waveform[:, :NUM_SAMPLES]

    return waveform # (1, NUM_SAMPLES)

def predict(model, audio_tensor, device=DEVICE, class_name=CLASS_NAME):
    audio_tensor = audio_tensor.to(device)

    with torch.no_grad():
        output_dict = model(audio_tensor)
        prob = output_dict["clipwise_output"] # 클래스별 확률 출력
        prob_np = prob.cpu().numpy()[0] # numpy 배열로 변환 (shape: [클래스 수])

        # 0.5를 기준으로 예측된 클래스 선택
        pred_tensor = (prob_np > 0.5).astype(int)
        pred_labels = [class_name[i] for i, p in enumerate(pred_tensor) if p == 1]

        # 어떤 클래스도 0.5 이상이 아니면 가장 확률 높은 클래스 하나 선택
        if not pred_labels:
            highest_prob_idx = int(np.argmax(prob_np))
            return [class_name[highest_prob_idx]], prob_np
        
        return pred_labels, prob_np # 예측된 라벨 리스트, 클래스별 확률 반환

In [11]:
model = torch.load(MODEL_PATH, map_location=torch.device(DEVICE), weights_only=False)
model.to(DEVICE)
model.eval()

Cnn14(
  (spectrogram_extractor): Spectrogram(
    (stft): STFT(
      (conv_real): Conv1d(1, 257, kernel_size=(512,), stride=(160,), bias=False)
      (conv_imag): Conv1d(1, 257, kernel_size=(512,), stride=(160,), bias=False)
    )
  )
  (logmel_extractor): LogmelFilterBank()
  (spec_augmenter): SpecAugmentation(
    (time_dropper): DropStripes()
    (freq_dropper): DropStripes()
  )
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (con

In [12]:
print("실시간 마이크 입력을 시작합니다.")

try:
    # 실시간 마이크 입력 루프
    while True:
        # 오디오를 실시간으로 마이크에서 녹음
        audio_np = sd.rec(frames=NUM_SAMPLES, # 녹음할 샘플 수 (1초 분량)
                          samplerate=SAMPLE_RATE, # 샘플링 레이트
                          channels=1, # 단일 채널 (모노)
                          dtype="float32" # float32 타입으로 녹음
                          )
        sd.wait() # 녹음 완료까지 대기

        audio_np = np.squeeze(audio_np, axis=-1) # 녹음된 오디오: (16000, 1) → (16000,)으로 reshape
        audio_tensor = preprocess(audio_np) # 전처리 → (1, 16000) 텐서로 변환
        pred_labels, class_prob = predict(model, audio_tensor) # 예측

        print(f"예측 결과: {', '.join(pred_labels)}")
        for i, name in enumerate(CLASS_NAME):
            print(f" - {name}: {class_prob[i]:.4f}")

except KeyboardInterrupt as e:
    print("\n실시간 마이크 입력을 종료합니다.\n", e)

실시간 마이크 입력을 시작합니다.
예측 결과: gas
 - danger: 0.0236
 - fire: 0.0106
 - gas: 0.3716
 - non: 0.1686
 - tsunami: 0.0022
예측 결과: non
 - danger: 0.0001
 - fire: 0.0000
 - gas: 0.0005
 - non: 0.9978
 - tsunami: 0.0000
예측 결과: non
 - danger: 0.0002
 - fire: 0.0000
 - gas: 0.0009
 - non: 0.9954
 - tsunami: 0.0000
예측 결과: non
 - danger: 0.0000
 - fire: 0.0000
 - gas: 0.0000
 - non: 0.9999
 - tsunami: 0.0000
예측 결과: non
 - danger: 0.0002
 - fire: 0.0001
 - gas: 0.0004
 - non: 0.9971
 - tsunami: 0.0000
예측 결과: non
 - danger: 0.0000
 - fire: 0.0002
 - gas: 0.0000
 - non: 0.9987
 - tsunami: 0.0000
예측 결과: non
 - danger: 0.0000
 - fire: 0.0001
 - gas: 0.0000
 - non: 0.9990
 - tsunami: 0.0000
예측 결과: non
 - danger: 0.0000
 - fire: 0.0000
 - gas: 0.0000
 - non: 0.9999
 - tsunami: 0.0000
예측 결과: non
 - danger: 0.0001
 - fire: 0.0002
 - gas: 0.0005
 - non: 0.9980
 - tsunami: 0.0001
예측 결과: non
 - danger: 0.0017
 - fire: 0.0017
 - gas: 0.0115
 - non: 0.9463
 - tsunami: 0.0004
예측 결과: non
 - danger: 0.0004
 - fire: 0.0